# Movielens 영화 추천 실습
- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있다. MovieLens 1M Dataset 사용을 권장합니다.
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.
- 별점을 시청횟수로 해석해서 생각하겠습니다.
- 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

# 1. 데이터 준비와 전처리

In [1]:
import pandas as pd
import os
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
import numpy as np

In [2]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
# 영화를 title로 입력할 것이기때문에 title이란 컬럼을 불러오기
ratings = pd.merge(ratings, movies[['title', 'movie_id']], on='movie_id', how='left')
ratings

,user_id,movie_id,counts,timestamp,title
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,978302109,James and the Giant Peach (1996)
2,1,914,3,978301968,My Fair Lady (1964)
3,1,3408,4,978300275,Erin Brockovich (2000)
4,1,2355,5,978824291,"Bug's Life, A (1998)"
...,...,...,...,...,...
836473,6040,1090,3,956715518,Platoon (1986)
836474,6040,1094,5,956704887,"Crying Game, The (1992)"
836475,6040,562,5,956704746,Welcome to the Dollhouse (1995)
836476,6040,1096,4,956715648,Sophie's Choice (1982)


# 2. 분석해 봅시다.

In [8]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [9]:
# ratings에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [10]:
# 가장 인기 있는 영화 30개(인기순)
movie_count = ratings.groupby('movie_id')['counts'].count()
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: counts, dtype: int64

# 3. 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [11]:
my_favorite = ['Toy Story (1995)' , 'Jumanji (1995)' ,'Grumpier Old Men (1995)' ,'Waiting to Exhale (1995)' ,'Father of the Bride Part II (1995)']

# 'jaekyung'이라는 user_id가 위 영화에 평점을 5점 줬다고 가정한다.
my_movies = pd.DataFrame({'user_id': ['jaekyung']*5, 'title': my_favorite, 'counts':[5]*5})

if not ratings.isin({'user_id':['jaekyung']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    ratings = ratings.append(my_movies)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,counts,timestamp,title
836473,6040,1090.0,3,956715518.0,Platoon (1986)
836474,6040,1094.0,5,956704887.0,"Crying Game, The (1992)"
836475,6040,562.0,5,956704746.0,Welcome to the Dollhouse (1995)
836476,6040,1096.0,4,956715648.0,Sophie's Choice (1982)
836477,6040,1097.0,4,956715569.0,E.T. the Extra-Terrestrial (1982)
0,jaekyung,NaN,5,NaN,Toy Story (1995)
1,jaekyung,NaN,5,NaN,Jumanji (1995)
2,jaekyung,NaN,5,NaN,Grumpier Old Men (1995)
3,jaekyung,NaN,5,NaN,Waiting to Exhale (1995)
4,jaekyung,NaN,5,NaN,Father of the Bride Part II (1995)


In [12]:
# 인덱싱
userid_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()

# user, movie indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(userid_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [13]:
# 인덱싱이 잘 되었는지 확인
print(user_to_idx['jaekyung'])    
print(movie_to_idx['Toy Story (1995)'])

6039
40


In [14]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie_id column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie_id column indexing Fail!!')

ratings

user_id column indexing OK!!
movie_id column indexing OK!!


,user_id,movie_id,counts,timestamp,title
0,0,0,5,978300760.0,One Flew Over the Cuckoo's Nest (1975)
1,0,1,3,978302109.0,James and the Giant Peach (1996)
2,0,2,3,978301968.0,My Fair Lady (1964)
3,0,3,4,978300275.0,Erin Brockovich (2000)
4,0,4,5,978824291.0,"Bug's Life, A (1998)"
...,...,...,...,...,...
0,6039,40,5,NaN,Toy Story (1995)
1,6039,513,5,NaN,Jumanji (1995)
2,6039,1862,5,NaN,Grumpier Old Men (1995)
3,6039,397,5,NaN,Waiting to Exhale (1995)


# 4. CSR matrix를 직접 만들어 봅시다.

In [15]:
# csr_matrix
num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['counts'], (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

# 5. ls_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [16]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=10, dtype=np.float32)

In [17]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [18]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/10 [00:00<?, ?it/s]

# 6. 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [19]:
jaekyung, toystory = user_to_idx['jaekyung'], movie_to_idx['Toy Story (1995)']
jaekyung_vector, toystory_vector = als_model.user_factors[jaekyung], als_model.item_factors[toystory]

In [20]:
# 선호하는 영화 내적하는 코드
np.dot(jaekyung_vector, toystory_vector)

0.47832468

In [21]:
platoon = movie_to_idx['Platoon (1986)']
platoon_vector = als_model.item_factors[platoon]
np.dot(jaekyung_vector, platoon_vector)

0.07949518

# 7. 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [22]:
# (id, 유사도 )
favorite_movie = 'Toy Story (1995)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(40, 1.0),
 (50, 0.76692843),
 (322, 0.63536924),
 (4, 0.6112116),
 (33, 0.6018759),
 (110, 0.54647624),
 (330, 0.519657),
 (20, 0.51593304),
 (10, 0.47957513),
 (255, 0.45252022),
 (160, 0.42695442),
 (34, 0.40697846),
 (478, 0.40024552),
 (32, 0.39848474),
 (126, 0.3892483)]

In [23]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Toy Story (1995)',
 'Toy Story 2 (1999)',
 'Babe (1995)',
 "Bug's Life, A (1998)",
 'Aladdin (1992)',
 'Groundhog Day (1993)',
 'Lion King, The (1994)',
 'Pleasantville (1998)',
 'Beauty and the Beast (1991)',
 "There's Something About Mary (1998)",
 'Forrest Gump (1994)',
 'Mulan (1998)',
 "Wayne's World (1992)",
 'Hercules (1997)',
 'Shakespeare in Love (1998)']

In [24]:
# 비슷한 영화 이름 검색 함수
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [26]:
get_similar_movie('Jumanji (1995)')

['Jumanji (1995)',
 'Indian in the Cupboard, The (1995)',
 'Hook (1991)',
 'Dragonheart (1996)',
 'Santa Clause, The (1994)',
 'Small Soldiers (1998)',
 'Flubber (1997)',
 'Space Jam (1996)',
 'Borrowers, The (1997)',
 'NeverEnding Story II: The Next Chapter, The (1990)']

# 8. 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [27]:
user = user_to_idx['jaekyung']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(50, 0.48127896),
 (4, 0.28137276),
 (545, 0.2637354),
 (596, 0.23230374),
 (60, 0.21196193),
 (1045, 0.21160471),
 (507, 0.2063233),
 (458, 0.20566532),
 (33, 0.19693688),
 (1982, 0.19557926),
 (322, 0.18738657),
 (330, 0.18446979),
 (32, 0.1835304),
 (1130, 0.18130201),
 (119, 0.17467853),
 (1514, 0.16824237),
 (341, 0.16801745),
 (173, 0.16064882),
 (38, 0.15919176),
 (476, 0.15652855)]

In [28]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Toy Story 2 (1999)',
 "Bug's Life, A (1998)",
 'Santa Clause, The (1994)',
 'Hook (1991)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Grumpy Old Men (1993)',
 'Home Alone (1990)',
 'Mask, The (1994)',
 'Aladdin (1992)',
 'Space Jam (1996)',
 'Babe (1995)',
 'Lion King, The (1994)',
 'Hercules (1997)',
 'Indian in the Cupboard, The (1995)',
 'Twister (1996)',
 'Six Days Seven Nights (1998)',
 'Mighty Ducks, The (1992)',
 'Dragonheart (1996)',
 'Sixth Sense, The (1999)',
 'Wrong Trousers, The (1993)']

# 9. 회고

처음에 영화 뒤에 연도를 안 적어 선호도가 너무 낮게 나왔는데, 그걸 모르고 계속 헤맸었다;; 연도를 적어주니 비슷한 영화들이 나와서 한숨 돌리고, 다행이라고 생각했다..ㅎㅎ 그리고 데이터 전처리 하는데 오타나, 잘못 입력값을 주면 데이터가 꼬여버려서 몇 번이고 restart를 했다.. 데이터 전처리할 때 정신 똑바로 차리고 해야겠다는 생각이 든 익스였다. 그리고 모델이 엄청 복잡하지 않은데도 그럭저럭 꽤나 학습하는 걸 보고 신기했다! 실제로 추천시스템은 더 복잡할텐데 얼른 레벨업해서 다양한 데이터도 만져보고 모델도 직접 쌓아보고싶다.